In [1]:
import pyspark as spark
import numpy as np
import pandas as pd
import sklearn
import geopandas as gpd 
from geopandas import GeoDataFrame
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


In [2]:
clean_crash = pd.read_csv('crash_clean.csv')

/Users/winsometang/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
clean_vehicles = pd.read_csv('clean_vehicles.csv')

In [4]:
clean_people = pd.read_csv('clean_people.csv')

/Users/winsometang/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (4,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
clean_crash.describe()

Unnamed: 0  POSTED_SPEED_LIMIT     CRASH_HOUR  CRASH_DAY_OF_WEEK  \
count  517282.00000       517282.000000  517282.000000      517282.000000   
mean   258640.50000           28.302120      13.242158           4.126892   
std    149326.59531            6.444674       5.506523           1.977442   
min         0.00000            0.000000       0.000000           1.000000   
25%    129320.25000           30.000000       9.000000           2.000000   
50%    258640.50000           30.000000      14.000000           4.000000   
75%    387960.75000           30.000000      17.000000           6.000000   
max    517281.00000           99.000000      23.000000           7.000000   

         CRASH_MONTH       LATITUDE      LONGITUDE      LANE_CNT  
count  517282.000000  514382.000000  514382.000000  1.989660e+05  
mean        6.573674      41.855046     -87.673046  1.333242e+01  
std         3.438811       0.336088       0.683169  2.961921e+03  
min         1.000000       0.000000     -87.934763  0.000000e+00  
25%         4.000000      41.782219     -87.721285  2.000000e+00  
50%         7.000000      41.875409     -87.673455  2.000000e+00  
75%        10.000000      41.924046     -87.632828  4.000000e+00  
max        12.000000      42.022780       0.000000  1.191625e+06

In [6]:
clean_people.describe()

Unnamed: 0            AGE  BAC_RESULT VALUE
count  1.142379e+06  815720.000000      1.142379e+06
mean   5.715878e+05      37.847115      2.140358e-04
std    3.299286e+05      17.106371      6.961254e-03
min    0.000000e+00    -177.000000      0.000000e+00
25%    2.858925e+05      25.000000      0.000000e+00
50%    5.716430e+05      35.000000      0.000000e+00
75%    8.573105e+05      50.000000      0.000000e+00
max    1.142962e+06     110.000000      1.000000e+00

In [7]:
clean_vehicles.describe()

Unnamed: 0  CRASH_UNIT_ID
count  1.057474e+06   1.057474e+06
mean   5.287365e+05   5.642652e+05
std    3.052666e+05   3.276157e+05
min    0.000000e+00   2.000000e+00
25%    2.643682e+05   2.790312e+05
50%    5.287365e+05   5.645995e+05
75%    7.931048e+05   8.490518e+05
max    1.057473e+06   1.129364e+06

In [8]:
clean_vehicles.head(10)

Unnamed: 0  CRASH_UNIT_ID  \
0           0         829999   
1           1         749947   
2           2         749949   
3           3         749950   
4           4         871921   
5           5         473653   
6           6         834812   
7           7         834813   
8           8         837380   
9           9         837381   

                                     CRASH_RECORD_ID              CRASH_DATE  \
0  24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...  01/22/2020 06:25:00 AM   
1  81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...  09/28/2019 03:30:00 AM   
2  81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...  09/28/2019 03:30:00 AM   
3  81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...  09/28/2019 03:30:00 AM   
4  af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...  04/13/2020 10:50:00 PM   
5  80f5b6eff349a72094c9c7a11745549caa8bd39b57cedd...  09/09/2018 08:50:00 AM   
6  ecf9e646f89a73ed3df0f373e4fbd18e91f5a595004019...  02/03/2020 03:45:00 PM   
7  ecf9e646f89a73ed3df0f373e4fbd18e91f5a595004019...  02/03/2020 03:45:00 PM   
8  c7c57a49274b3aa953d0b76346f68e116ed8c3d72a683a...  02/06/2020 12:00:00 AM   
9  c7c57a49274b3aa953d0b76346f68e116ed8c3d72a683a...  02/06/2020 12:00:00 AM   

                               MAKE     MODEL VEHICLE_TYPE VEHICLE_DEFECT  \
0                          INFINITI   UNKNOWN    PASSENGER           NONE   
1                             HONDA     CIVIC    PASSENGER        UNKNOWN   
2                            TOYOTA     YARIS   UNKNOWN/NA           NONE   
3  GENERAL MOTORS CORPORATION (GMC)    SIERRA   UNKNOWN/NA        UNKNOWN   
4                             BUICK    ENCORE    PASSENGER           NONE   
5               LINCOLN-CONTINENTAL       MKT    PASSENGER           NONE   
6                             HONDA       CRV    PASSENGER           NONE   
7                            SUBARU  FORESTER    PASSENGER           NONE   
8                           UNKNOWN   UNKNOWN   UNKNOWN/NA        UNKNOWN   
9                          INFINITI   UNKNOWN    PASSENGER        UNKNOWN   

  TRAVEL_DIRECTION        MANEUVER FIRST_CONTACT_POINT FIRE_I  
0                N  STRAIGHT AHEAD               FRONT      N  
1                N  STRAIGHT AHEAD               FRONT      N  
2                N          PARKED                ROOF      N  
3                N          PARKED                ROOF      N  
4                W  STRAIGHT AHEAD         FRONT-RIGHT      N  
5                S          PARKED          FRONT-LEFT      N  
6                W  STRAIGHT AHEAD               OTHER      N  
7                S  STRAIGHT AHEAD                ROOF      N  
8          UNKNOWN      UNKNOWN/NA             UNKNOWN      N  
9          UNKNOWN          PARKED   TOTAL (ALL AREAS)      N

In [9]:
clean_crash.head(10)

Unnamed: 0                                    CRASH_RECORD_ID  \
0           0  4fd0a3e0897b3335b94cd8d5b2d2b350eb691add56c62d...   
1           1  009e9e67203442370272e1a13d6ee51a4155dac65e583d...   
2           2  ee9283eff3a55ac50ee58f3d9528ce1d689b1c4180b4c4...   
3           3  f8960f698e870ebdc60b521b2a141a5395556bc3704191...   
4           4  8eaa2678d1a127804ee9b8c35ddf7d63d913c14eda61d6...   
5           5  00e47f189660cd8ba1e85fc63061bf1d8465184393f134...   
6           6  0126747fc9ffc0edc9a38abb83d80034f897db0f739eef...   
7           7  f636d4a51a88015ac89031159b1f1952b8d92e49d11aeb...   
8           8  76aabcf7c2219a5c90259c96fe94b33834ddb53e0dbcd3...   
9           9  9c974548026c1b962569040bd8fa08ae643ffc28c15ebd...   

               CRASH_DATE  POSTED_SPEED_LIMIT WEATHER_CONDITION  \
0  07/10/2019 05:56:00 PM                  35             CLEAR   
1  06/30/2017 04:00:00 PM                  35             CLEAR   
2  07/10/2020 10:25:00 AM                  30             CLEAR   
3  07/11/2020 01:00:00 AM                  30             CLEAR   
4  07/08/2020 02:00:00 PM                  20             CLEAR   
5  03/21/2019 10:50:00 PM                  30             CLEAR   
6  03/26/2018 02:23:00 PM                  35             CLEAR   
7  07/10/2020 10:20:00 PM                  30             CLEAR   
8  07/09/2020 05:06:00 PM                  10             CLEAR   
9  06/29/2020 05:55:00 PM                  10   CLOUDY/OVERCAST   

       LIGHTING_CONDITION              FIRST_CRASH_TYPE           ALIGNMENT  \
0                DAYLIGHT                       TURNING  STRAIGHT AND LEVEL   
1                DAYLIGHT                       TURNING  STRAIGHT AND LEVEL   
2                DAYLIGHT                      REAR END  STRAIGHT AND LEVEL   
3                DARKNESS          PARKED MOTOR VEHICLE  STRAIGHT AND LEVEL   
4                DAYLIGHT          PARKED MOTOR VEHICLE  STRAIGHT AND LEVEL   
5  DARKNESS, LIGHTED ROAD                       TURNING  STRAIGHT AND LEVEL   
6                DAYLIGHT          PARKED MOTOR VEHICLE  STRAIGHT AND LEVEL   
7  DARKNESS, LIGHTED ROAD  SIDESWIPE OPPOSITE DIRECTION  STRAIGHT AND LEVEL   
8                DAYLIGHT          PARKED MOTOR VEHICLE  STRAIGHT AND LEVEL   
9                    DAWN          PARKED MOTOR VEHICLE  STRAIGHT AND LEVEL   

  ROADWAY_SURFACE_COND ROAD_DEFECT  ... INJURIES_TOTAL INJURIES_FATAL  \
0                  DRY  NO DEFECTS  ...            0.0            0.0   
1                  DRY  NO DEFECTS  ...            0.0            0.0   
2                  DRY  NO DEFECTS  ...            0.0            0.0   
3                  DRY  NO DEFECTS  ...            0.0            0.0   
4                  DRY  NO DEFECTS  ...            0.0            0.0   
5                  DRY  NO DEFECTS  ...            0.0            0.0   
6                  DRY  NO DEFECTS  ...            0.0            0.0   
7                  DRY  NO DEFECTS  ...            1.0            0.0   
8              UNKNOWN     UNKNOWN  ...            0.0            0.0   
9                  DRY  NO DEFECTS  ...            0.0            0.0   

  CRASH_HOUR CRASH_DAY_OF_WEEK CRASH_MONTH   LATITUDE  LONGITUDE  \
0         17                 4           7  41.919664 -87.773288   
1         16                 6           6  41.741804 -87.740954   
2         10                 6           7  41.773456 -87.585022   
3          1                 7           7  41.802119 -87.622115   
4         14                 4           7        NaN        NaN   
5         22                 5           3  41.741804 -87.740954   
6         14                 2           3  41.953647 -87.732082   
7         22                 6           7  41.880386 -87.725807   
8         17                 5           7        NaN        NaN   
9         17                 2           6        NaN        NaN   

                                   LOCATION                  TRAFFICWAY_TYPE  \
0  POINT (-87

In [10]:
clean_vehicles.head(10)

Unnamed: 0  CRASH_UNIT_ID  \
0           0         829999   
1           1         749947   
2           2         749949   
3           3         749950   
4           4         871921   
5           5         473653   
6           6         834812   
7           7         834813   
8           8         837380   
9           9         837381   

                                     CRASH_RECORD_ID              CRASH_DATE  \
0  24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...  01/22/2020 06:25:00 AM   
1  81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...  09/28/2019 03:30:00 AM   
2  81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...  09/28/2019 03:30:00 AM   
3  81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...  09/28/2019 03:30:00 AM   
4  af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...  04/13/2020 10:50:00 PM   
5  80f5b6eff349a72094c9c7a11745549caa8bd39b57cedd...  09/09/2018 08:50:00 AM   
6  ecf9e646f89a73ed3df0f373e4fbd18e91f5a595004019...  02/03/2020 03:45:00 PM   
7  ecf9e646f89a73ed3df0f373e4fbd18e91f5a595004019...  02/03/2020 03:45:00 PM   
8  c7c57a49274b3aa953d0b76346f68e116ed8c3d72a683a...  02/06/2020 12:00:00 AM   
9  c7c57a49274b3aa953d0b76346f68e116ed8c3d72a683a...  02/06/2020 12:00:00 AM   

                               MAKE     MODEL VEHICLE_TYPE VEHICLE_DEFECT  \
0                          INFINITI   UNKNOWN    PASSENGER           NONE   
1                             HONDA     CIVIC    PASSENGER        UNKNOWN   
2                            TOYOTA     YARIS   UNKNOWN/NA           NONE   
3  GENERAL MOTORS CORPORATION (GMC)    SIERRA   UNKNOWN/NA        UNKNOWN   
4                             BUICK    ENCORE    PASSENGER           NONE   
5               LINCOLN-CONTINENTAL       MKT    PASSENGER           NONE   
6                             HONDA       CRV    PASSENGER           NONE   
7                            SUBARU  FORESTER    PASSENGER           NONE   
8                           UNKNOWN   UNKNOWN   UNKNOWN/NA        UNKNOWN   
9                          INFINITI   UNKNOWN    PASSENGER        UNKNOWN   

  TRAVEL_DIRECTION        MANEUVER FIRST_CONTACT_POINT FIRE_I  
0                N  STRAIGHT AHEAD               FRONT      N  
1                N  STRAIGHT AHEAD               FRONT      N  
2                N          PARKED                ROOF      N  
3                N          PARKED                ROOF      N  
4                W  STRAIGHT AHEAD         FRONT-RIGHT      N  
5                S          PARKED          FRONT-LEFT      N  
6                W  STRAIGHT AHEAD               OTHER      N  
7                S  STRAIGHT AHEAD                ROOF      N  
8          UNKNOWN      UNKNOWN/NA             UNKNOWN      N  
9          UNKNOWN          PARKED   TOTAL (ALL AREAS)      N

In [11]:
clean_people.shape

(1142379, 16)

In [12]:
clean_vehicles.shape

(1057474, 12)

In [13]:
clean_crash.shape

(517282, 27)

In [14]:
mergeddf=clean_people.merge(clean_vehicles,on='CRASH_RECORD_ID').merge(clean_crash,on='CRASH_RECORD_ID')

In [15]:
mergeddf.describe()

Unnamed: 0_x           AGE  BAC_RESULT VALUE  Unnamed: 0_y  \
count  2.412776e+06  1.727995e+06      2.412776e+06  2.412776e+06   
mean   5.702466e+05  3.773590e+01      2.414066e-04  5.250276e+05   
std    3.327499e+05  1.710756e+01      7.356951e-03  3.068851e+05   
min    0.000000e+00 -1.770000e+02      0.000000e+00  0.000000e+00   
25%    2.797910e+05  2.500000e+01      0.000000e+00  2.577478e+05   
50%    5.696345e+05  3.500000e+01      0.000000e+00  5.244035e+05   
75%    8.599260e+05  5.000000e+01      0.000000e+00  7.905772e+05   
max    1.142962e+06  1.100000e+02      1.000000e+00  1.057473e+06   

       CRASH_UNIT_ID    Unnamed: 0  POSTED_SPEED_LIMIT    CRASH_HOUR  \
count   2.412776e+06  2.412776e+06        2.412776e+06  2.412776e+06   
mean    5.676385e+05  2.542429e+05        2.886386e+01  1.350763e+01   
std     3.259501e+05  1.514699e+05        5.905738e+00  5.394394e+00   
min     2.000000e+00  0.000000e+00        0.000000e+00  0.000000e+00   
25%     2.850108e+05  1.219690e+05        3.000000e+01  1.000000e+01   
50%     5.697720e+05  2.540475e+05        3.000000e+01  1.400000e+01   
75%     8.496552e+05  3.854432e+05        3.000000e+01  1.700000e+01   
max     1.129364e+06  5.172810e+05        9.900000e+01  2.300000e+01   

       CRASH_DAY_OF_WEEK   CRASH_MONTH      LATITUDE     LONGITUDE  \
count       2.412776e+06  2.412776e+06  2.399324e+06  2.399324e+06   
mean        4.148680e+00  6.590377e+00  4.185222e+01 -8.766854e+01   
std         1.988100e+00  3.423331e+00  4.496020e-01  9.267125e-01   
min         1.000000e+00  1.000000e+00  0.000000e+00 -8.793476e+01   
25%         2.000000e+00  4.000000e+00  4.178215e+01 -8.772162e+01   
50%         4.000000e+00  7.000000e+00  4.187489e+01 -8.767357e+01   
75%         6.000000e+00  1.000000e+01  4.192198e+01 -8.763256e+01   
max         7.000000e+00  1.200000e+01  4.202278e+01  0.000000e+00   

           LANE_CNT  
count  9.296760e+05  
mean   9.349354e+00  
std    2.110810e+03  
min    0.000000e+00  
25%    2.000000e+00  
50%    2.000000e+00  
75%    4.000000e+00  
max    1.191625e+06

In [16]:
mergeddf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2412776 entries, 0 to 2412775
Data columns (total 53 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   Unnamed: 0_x             int64  
 1   PERSON_ID                object 
 2   PERSON_TYPE              object 
 3   CRASH_RECORD_ID          object 
 4   VEHICLE_ID               object 
 5   CRASH_DATE_x             object 
 6   SEX                      object 
 7   AGE                      float64
 8   EJECTION                 object 
 9   INJURY_CLASSIFICATION    object 
 10  AIRBAG_DEPLOYED          object 
 11  DRIVER_ACTION            object 
 12  PEDPEDAL_ACTION          object 
 13  PHYSICAL_CONDITION       object 
 14  BAC_RESULT VALUE         float64
 15  SAFETY_EQUIPMENT         object 
 16  Unnamed: 0_y             int64  
 17  CRASH_UNIT_ID            int64  
 18  CRASH_DATE_y             object 
 19  MAKE                     object 
 20  MODEL                    object 
 21  VEHICLE_

In [17]:
mergeddf.shape

(2412776, 53)

In [18]:
mergeddf['MOST_SEVERE_INJURY'].value_counts()


NO INDICATION OF INJURY     1975459
NONINCAPACITATING INJURY     239348
REPORTED, NOT EVIDENT        136521
INCAPACITATING INJURY         57052
FATAL                          4396
Name: MOST_SEVERE_INJURY, dtype: int64

In [19]:
mergeddf['INJURIES_FATAL'].value_counts()

0.0    2408380
1.0       3991
2.0        297
3.0         68
4.0         40
Name: INJURIES_FATAL, dtype: int64

In [20]:
mergeddf["FATAL"]= pd.to_numeric(mergeddf["INJURIES_FATAL"])


In [21]:
mergeddf['FATAL'].value_counts()



0.0    2408380
1.0       3991
2.0        297
3.0         68
4.0         40
Name: FATAL, dtype: int64

In [22]:
mergeddf.isnull().sum()

Unnamed: 0_x                     0
PERSON_ID                        0
PERSON_TYPE                      0
CRASH_RECORD_ID                  0
VEHICLE_ID                       0
CRASH_DATE_x                     0
SEX                              0
AGE                         684781
EJECTION                         0
INJURY_CLASSIFICATION            0
AIRBAG_DEPLOYED                  0
DRIVER_ACTION               501731
PEDPEDAL_ACTION                  0
PHYSICAL_CONDITION               0
BAC_RESULT VALUE                 0
SAFETY_EQUIPMENT                 0
Unnamed: 0_y                     0
CRASH_UNIT_ID                    0
CRASH_DATE_y                     0
MAKE                             0
MODEL                            0
VEHICLE_TYPE                     0
VEHICLE_DEFECT                   0
TRAVEL_DIRECTION                 0
MANEUVER                         0
FIRST_CONTACT_POINT              0
FIRE_I                           0
Unnamed: 0                       0
CRASH_DATE          

In [23]:
mergeddf.FATAL[mergeddf.FATAL > 0] = 1
mergeddf.FATAL[mergeddf.FATAL == 0] = 0

/var/folders/nt/rg8_jkq90v77r8l8dc_9wl0m0000gn/T/ipykernel_10118/2906185067.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mergeddf.FATAL[mergeddf.FATAL > 0] = 1
/var/folders/nt/rg8_jkq90v77r8l8dc_9wl0m0000gn/T/ipykernel_10118/2906185067.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mergeddf.FATAL[mergeddf.FATAL == 0] = 0


In [24]:
mergeddf.FATAL.value_counts()



0.0    2408380
1.0       4396
Name: FATAL, dtype: int64

In [25]:
model_data =mergeddf.drop(columns=['Unnamed: 0_x','Unnamed: 0','CRASH_DATE_x','CRASH_DATE_y','CRASH_DATE_y','Unnamed: 0_y','PERSON_ID','CRASH_RECORD_ID', 'VEHICLE_ID','CRASH_DATE','INJURIES_FATAL','LATITUDE', 'LONGITUDE', 'LOCATION','MOST_SEVERE_INJURY','INJURIES_TOTAL','INJURY_CLASSIFICATION'])


In [26]:
ten_samp = model_data[['EJECTION','AIRBAG_DEPLOYED',
       'SAFETY_EQUIPMENT', 'CRASH_UNIT_ID', 'MAKE',
       'MODEL','FIRST_CONTACT_POINT', 'FIRE_I',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION','ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'CRASH_TYPE','FATAL']]


In [27]:
model_data.columns

Index(['PERSON_TYPE', 'SEX', 'AGE', 'EJECTION', 'AIRBAG_DEPLOYED',
       'DRIVER_ACTION', 'PEDPEDAL_ACTION', 'PHYSICAL_CONDITION',
       'BAC_RESULT VALUE', 'SAFETY_EQUIPMENT', 'CRASH_UNIT_ID', 'MAKE',
       'MODEL', 'VEHICLE_TYPE', 'VEHICLE_DEFECT', 'TRAVEL_DIRECTION',
       'MANEUVER', 'FIRST_CONTACT_POINT', 'FIRE_I', 'POSTED_SPEED_LIMIT',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE',
       'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'CRASH_TYPE',
       'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_DIRECTION',
       'BEAT_OF_OCCURRENCE', 'WORK_ZONE_I', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK',
       'CRASH_MONTH', 'TRAFFICWAY_TYPE', 'LANE_CNT', 'FATAL'],
      dtype='object')

In [28]:
sample_df = model_data.sample(n = 50000)
ten_rand = ten_samp.sample(n = 50000)

In [29]:
sample_df.columns

Index(['PERSON_TYPE', 'SEX', 'AGE', 'EJECTION', 'AIRBAG_DEPLOYED',
       'DRIVER_ACTION', 'PEDPEDAL_ACTION', 'PHYSICAL_CONDITION',
       'BAC_RESULT VALUE', 'SAFETY_EQUIPMENT', 'CRASH_UNIT_ID', 'MAKE',
       'MODEL', 'VEHICLE_TYPE', 'VEHICLE_DEFECT', 'TRAVEL_DIRECTION',
       'MANEUVER', 'FIRST_CONTACT_POINT', 'FIRE_I', 'POSTED_SPEED_LIMIT',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE',
       'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'CRASH_TYPE',
       'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_DIRECTION',
       'BEAT_OF_OCCURRENCE', 'WORK_ZONE_I', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK',
       'CRASH_MONTH', 'TRAFFICWAY_TYPE', 'LANE_CNT', 'FATAL'],
      dtype='object')

In [30]:
ten_rand.columns

Index(['EJECTION', 'AIRBAG_DEPLOYED', 'SAFETY_EQUIPMENT', 'CRASH_UNIT_ID',
       'MAKE', 'MODEL', 'FIRST_CONTACT_POINT', 'FIRE_I', 'WEATHER_CONDITION',
       'LIGHTING_CONDITION', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT',
       'CRASH_TYPE', 'FATAL'],
      dtype='object')

In [31]:
sample_dummies = pd.get_dummies(sample_df, columns = ['PERSON_TYPE', 'SEX', 'AGE', 'EJECTION', 'AIRBAG_DEPLOYED',
       'DRIVER_ACTION', 'PEDPEDAL_ACTION', 'PHYSICAL_CONDITION',
       'BAC_RESULT VALUE', 'SAFETY_EQUIPMENT', 'CRASH_UNIT_ID', 'MAKE',
       'MODEL', 'VEHICLE_TYPE', 'VEHICLE_DEFECT', 'TRAVEL_DIRECTION',
       'MANEUVER', 'FIRST_CONTACT_POINT', 'FIRE_I', 'POSTED_SPEED_LIMIT',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE',
       'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'CRASH_TYPE',
       'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 'STREET_DIRECTION',
       'BEAT_OF_OCCURRENCE', 'WORK_ZONE_I', 'CRASH_HOUR', 'CRASH_DAY_OF_WEEK',
       'CRASH_MONTH', 'TRAFFICWAY_TYPE', 'LANE_CNT'])


In [32]:
ten_samp = pd.get_dummies(sample_df, columns = ['EJECTION', 'AIRBAG_DEPLOYED',
       'SAFETY_EQUIPMENT', 'CRASH_UNIT_ID', 'MAKE',
       'MODEL','FIRST_CONTACT_POINT', 'FIRE_I',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION','ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'CRASH_TYPE'])

In [33]:
sample_df["FATAL"].value_counts()




0.0    49912
1.0       88
Name: FATAL, dtype: int64

In [ ]:
y = sample_df['FATAL']
X = sample_dummies
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
model = LogisticRegression()
model.fit(X_train,y_train)

prediction = model.predict(X_test)
score = model.score(X_test,y_test)




In [ ]:
print(prediction)
print(score)

In [ ]:
ten_samp.columns


In [ ]:
y = ten_rand['FATAL']
X = ten_samp.drop(columns = ['PERSON_TYPE', 'SEX', 'AGE', 'DRIVER_ACTION', 'PEDPEDAL_ACTION',
       'PHYSICAL_CONDITION', 'BAC_RESULT VALUE', 'VEHICLE_TYPE',
       'VEHICLE_DEFECT', 'TRAVEL_DIRECTION'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)
model = LogisticRegression(njobs = -1)
model.fit(X_train,y_train)

samp_pred = model.predict(X_test)
samp_score = model.score(X_test,y_test)

In [ ]:
print(samp_score)